# COMP5318 - Machine Learning and Data Mining: Assignment 1

In [63]:
import pandas as pd
import os
print(os.listdir("./Input/train"))
pd.set_option('display.max_columns', 10)

from IPython.display import set_matplotlib_formats, display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import graphviz

%matplotlib inline

['train.csv']


In [64]:
# train.csv including feature and label using for training model.
data_train_df = pd.read_csv('./Input/train/train.csv') 
# data_train_df = data_train_df.sample(frac = 0.5)

In [65]:
# Selecting input feature
data_train_feature = data_train_df.loc[:, "v1":"v784"].to_numpy()

# Selecting output lable 
data_train_label = data_train_df.label.to_numpy()

## DATA PRE-PROCESSING FOR TRAINING DATA

In [66]:
pd.DataFrame(data_train_feature)

,0,1,2,3,4,...,779,780,781,782,783
0,0,0,0,0,1,...,0,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,1,0,...,61,0,0,0,0
4,0,0,0,0,0,...,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
29995,0,0,0,0,0,...,0,0,0,0,0
29996,0,0,0,0,0,...,0,0,0,0,0
29997,0,0,0,0,0,...,0,0,0,0,0
29998,0,0,0,0,0,...,0,0,0,0,0


In [67]:
X_train, X_test, y_train, y_test = train_test_split(
    data_train_feature, data_train_label, random_state=0)

In [68]:
# Normalisation
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm  = scaler.transform(X_test)

pd.DataFrame(X_train_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,...,0.709804,0.156,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
22495,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
22496,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
22497,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0
22498,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000,0.0,0.0,0.0


In [69]:
pca = PCA(n_components=0.95).fit(X_train_norm)

X_train_pca = pca.transform(X_train_norm)
X_test_pca = pca.transform(X_test_norm)

pd.DataFrame(X_train_pca)

,0,1,2,3,4,...,181,182,183,184,185
0,-0.394598,-5.686051,0.913325,-1.125150,-1.570830,...,-0.099306,-0.032330,0.028308,0.091482,0.049649
1,1.143164,-5.712517,1.264873,-1.803338,-1.362444,...,-0.027480,0.034489,-0.139472,-0.098686,0.022165
2,-6.141011,-0.336818,-0.597506,2.178537,-0.415588,...,-0.282094,-0.068421,-0.211602,-0.170289,-0.044729
3,-5.601226,2.431388,-0.185770,-2.443250,1.362475,...,0.105897,0.103251,-0.175070,0.127198,-0.043023
4,8.367414,3.090589,-2.711405,-0.902178,-0.412654,...,0.089727,-0.084119,-0.128825,0.014943,0.071792
...,...,...,...,...,...,...,...,...,...,...,...
22495,0.867325,-4.352062,1.205116,-0.741638,-1.467599,...,-0.126527,0.202405,-0.034824,-0.120592,-0.268840
22496,-6.117047,0.943566,-1.035591,-0.649070,0.617476,...,0.046579,0.007056,0.303242,0.098692,-0.215452
22497,-1.023785,-1.232009,-2.922173,1.696158,0.814303,...,0.062033,-0.074712,0.108450,-0.182543,0.119338
22498,0.604955,-6.065121,1.629911,-2.197434,-2.092257,...,0.005359,-0.026001,0.096928,-0.004058,-0.013008


## DATA PRE-PROCESSING FOR TESTING DATA

In [70]:
# test_input.csv includes 5000 samples used for label prediction. Test samples do not have labels.
data_test_df = pd.read_csv('./Input/test/test_input.csv', index_col=0) 

In [71]:
# Data Normalisation
output_test_norm = scaler.transform(data_test_df.to_numpy())

pd.DataFrame(output_test_norm)

,0,1,2,3,4,...,779,780,781,782,783
0,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
1,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
2,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
3,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4996,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4997,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0
4998,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.000,0.000000,0.0,0.0


In [72]:
# Dimension Reduction
output_test_pca = pca.transform(output_test_norm)

pd.DataFrame(output_test_pca)

,0,1,2,3,4,...,181,182,183,184,185
0,1.978747,-5.457940,1.946917,-1.959688,-1.010971,...,0.085449,-0.056032,0.078949,-0.044243,0.028917
1,1.923139,-5.680224,1.295473,-1.416505,-0.594287,...,0.008092,0.000689,-0.078681,0.048155,0.018189
2,0.472941,-5.856364,1.597844,-2.046583,-2.592412,...,0.081588,-0.042409,0.050636,-0.034319,0.192345
3,-0.375038,-3.365100,-0.763618,1.793366,0.032026,...,-0.123256,-0.029422,0.188747,0.047197,0.006229
4,-1.540833,-2.601513,-0.900526,0.412424,0.679410,...,-0.109269,-0.015584,-0.198439,0.018514,0.003453
...,...,...,...,...,...,...,...,...,...,...,...
4995,2.739363,-3.216671,0.817188,0.588171,-0.349807,...,0.132634,0.027308,0.075524,0.189554,0.015515
4996,2.462737,5.937648,-0.364203,1.201229,3.887794,...,-0.119567,-0.204678,0.062225,-0.096710,0.066106
4997,-5.694177,-0.599542,-1.165802,3.353452,-0.944159,...,-0.277260,-0.061213,0.108531,0.323516,0.473816
4998,7.058567,1.491336,0.489352,1.188750,0.217547,...,-0.014977,0.080540,-0.079943,-0.083805,-0.055174


## KNN

In [73]:
print("X_train_pca shape:", X_train_pca.shape)
print("y_train shape:", y_train.shape)

X_train_pca shape: (22500, 186)
y_train shape: (22500,)


In [74]:
print("X_test_pca shape:", X_test_pca.shape)
print("y_test shape:", y_test.shape)

X_test_pca shape: (7500, 186)
y_test shape: (7500,)


In [75]:
param_grid = {'n_neighbors': [1, 3, 5, 11, 15], 'p': [1, 2]}
print("Parameter grid:\n{}".format(param_grid))

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, return_train_score=True, n_jobs=-1)

grid_search.fit(X_train_pca, y_train)

print("Test set score: {:.2f}".format(grid_search.score(X_test_pca, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

best_n = grid_search.best_params_['n_neighbors']
best_p = grid_search.best_params_['p']
knn = KNeighborsClassifier(n_neighbors=best_n, p=best_p)
knn.fit(X_train_pca, y_train)

Parameter grid:
{'n_neighbors': [1, 3, 5, 11, 15], 'p': [1, 2]}
Test set score: 0.85
Best parameters: {'n_neighbors': 5, 'p': 2}
Best cross-validation score: 0.85
Best estimator:
KNeighborsClassifier()


KNeighborsClassifier()

In [76]:
# KNN Predictions
predictions = []

for i in output_test_pca:
    prediction = knn.predict([list(i)])
    predictions.append(prediction[0])

predictions

[1,
 1,
 1,
 0,
 6,
 6,
 1,
 2,
 9,
 5,
 9,
 9,
 8,
 4,
 4,
 3,
 0,
 2,
 5,
 3,
 0,
 4,
 8,
 7,
 0,
 2,
 9,
 6,
 5,
 5,
 8,
 7,
 8,
 3,
 7,
 7,
 6,
 6,
 3,
 3,
 8,
 0,
 2,
 6,
 1,
 0,
 4,
 7,
 0,
 0,
 2,
 8,
 9,
 7,
 3,
 0,
 7,
 1,
 3,
 9,
 8,
 0,
 9,
 0,
 5,
 0,
 2,
 2,
 9,
 6,
 7,
 2,
 3,
 9,
 9,
 8,
 5,
 1,
 1,
 7,
 4,
 6,
 7,
 8,
 5,
 7,
 2,
 0,
 7,
 6,
 3,
 4,
 0,
 3,
 3,
 9,
 4,
 5,
 0,
 8,
 7,
 9,
 4,
 3,
 1,
 4,
 6,
 9,
 2,
 9,
 2,
 3,
 9,
 3,
 4,
 5,
 0,
 7,
 8,
 3,
 6,
 2,
 3,
 7,
 9,
 8,
 6,
 0,
 3,
 0,
 2,
 1,
 5,
 6,
 4,
 6,
 7,
 7,
 3,
 8,
 8,
 6,
 6,
 1,
 8,
 1,
 9,
 4,
 1,
 3,
 3,
 4,
 4,
 2,
 8,
 8,
 2,
 0,
 1,
 2,
 3,
 0,
 9,
 2,
 4,
 6,
 2,
 7,
 7,
 4,
 8,
 7,
 3,
 1,
 2,
 1,
 3,
 8,
 7,
 4,
 7,
 4,
 6,
 5,
 9,
 0,
 6,
 6,
 5,
 9,
 2,
 8,
 9,
 4,
 0,
 1,
 7,
 9,
 0,
 9,
 7,
 5,
 0,
 7,
 7,
 4,
 9,
 7,
 0,
 2,
 4,
 5,
 7,
 2,
 4,
 7,
 1,
 5,
 0,
 5,
 9,
 2,
 2,
 0,
 1,
 5,
 1,
 5,
 7,
 7,
 9,
 5,
 7,
 7,
 6,
 0,
 9,
 8,
 3,
 0,
 0,
 7,
 9,
 3,
 0,
 8,
 6,
 3,
 1,
 3,


In [77]:
output_df = pd.DataFrame(predictions, columns = ['label'])
output_df.to_csv('test_output.csv', sep=",", float_format='%d', index_label="id")

## LOGISTIC REGRESSION

In [78]:
logreg = LogisticRegression(max_iter = 5000)
logreg.fit(X_train_pca, y_train)
print("Accuracy on trainig set:", logreg.score(X_train_pca, y_train)*100)
print("Accuracy on test set:", logreg.score(X_test_pca, y_test)*100)

Accuracy on trainig set: 86.94666666666667
Accuracy on test set: 84.41333333333333


## NAIVE BAYES

In [79]:
nb = GaussianNB()
nb.fit(X_train_pca, y_train)
y_pred = nb.predict(X_test_pca)
print("Accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy on test set: 0.740


In [81]:
# Parameter Tuning 
param_grid_nb = { 'var_smoothing': np.logspace(0,-9, num=100) }

nbModel_grid = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
nbModel_grid.fit(X_train_pca, y_train)

print("Test set score: ", nbModel_grid.score(X_test_pca, y_test))
print("Best parameters: {}".format(nbModel_grid.best_params_))
print("Best cross-validation score: ", nbModel_grid.best_score_)

ideal_var_smoothing = nbModel_grid.best_params_['var_smoothing']
nb = GaussianNB(var_smoothing=ideal_var_smoothing)
nb.fit(X_train_pca, y_train)
y_pred = nb.predict(X_test_pca)
print("\nAccuracy on test set after parameter tuning: {:.3f}".format(accuracy_score(y_test, y_pred)))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Test set score:  0.7424
Best parameters: {'var_smoothing': 0.0001519911082952933}
Best cross-validation score:  0.7496

Accuracy on test set after parameter tuning: 0.742


## DECISION TREE

In [82]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=12, random_state=42)
tree.fit(X_train_pca, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=12, random_state=42)

In [83]:
print("Accuracy on training set: {:.3f}".format(tree.score(X_train_pca, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test_pca, y_test)))

Accuracy on training set: 0.914
Accuracy on test set: 0.757


## SVM

In [84]:
# SVM with polynomial kernel
poly_svm = SVC(kernel="poly", degree=2) #polynomial kernel with degree 2
poly_svm.fit(X_train_pca, y_train)
y_pred = poly_svm.predict(X_test_pca)
print("SVM with polynomial kernel - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

SVM with polynomial kernel - accuracy on test set: 0.868


In [85]:
# SVM with Linear Kernel
lin_svm = SVC(kernel="linear")
lin_svm.fit(X_train_pca, y_train)

y_pred = lin_svm.predict(X_test_pca)
print("Linear SVM - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

Linear SVM - accuracy on test set: 0.847


In [86]:
# SVM with RBF kernel
rbf_svm = SVC(kernel="rbf", gamma="auto")
rbf_svm.fit(X_train_pca, y_train)
y_pred = rbf_svm.predict(X_test_pca)
print("SVM with RBF kernel - accuracy on test set: {:.3f}".format(accuracy_score(y_test, y_pred)))

SVM with RBF kernel - accuracy on test set: 0.856


In [89]:
# Poly SVM Predictions 
predictions = []

for i in output_test_pca:
    prediction = poly_svm.predict([list(i)])
    predictions.append(prediction[0])

predictions

[1,
 1,
 1,
 0,
 3,
 6,
 1,
 2,
 9,
 5,
 9,
 9,
 8,
 4,
 4,
 3,
 0,
 2,
 5,
 3,
 6,
 4,
 8,
 7,
 3,
 2,
 9,
 6,
 5,
 5,
 8,
 7,
 8,
 3,
 7,
 7,
 6,
 6,
 3,
 3,
 8,
 0,
 2,
 6,
 3,
 2,
 4,
 5,
 0,
 0,
 4,
 8,
 9,
 7,
 3,
 0,
 7,
 1,
 3,
 9,
 8,
 6,
 5,
 0,
 5,
 0,
 2,
 2,
 9,
 0,
 7,
 2,
 3,
 9,
 9,
 8,
 5,
 1,
 1,
 7,
 2,
 6,
 7,
 8,
 5,
 7,
 2,
 0,
 7,
 6,
 3,
 4,
 0,
 3,
 3,
 9,
 4,
 5,
 0,
 8,
 7,
 9,
 4,
 3,
 1,
 4,
 6,
 7,
 2,
 9,
 2,
 3,
 9,
 3,
 4,
 5,
 6,
 7,
 8,
 3,
 6,
 2,
 3,
 7,
 9,
 8,
 3,
 0,
 3,
 6,
 4,
 1,
 5,
 6,
 4,
 6,
 7,
 7,
 3,
 8,
 8,
 8,
 6,
 1,
 8,
 1,
 9,
 2,
 1,
 3,
 3,
 4,
 4,
 2,
 8,
 8,
 6,
 6,
 1,
 2,
 3,
 0,
 9,
 2,
 4,
 6,
 2,
 7,
 7,
 4,
 8,
 7,
 3,
 1,
 2,
 1,
 3,
 6,
 7,
 2,
 7,
 4,
 6,
 5,
 9,
 0,
 6,
 2,
 5,
 9,
 2,
 8,
 9,
 6,
 0,
 1,
 7,
 9,
 0,
 9,
 7,
 5,
 0,
 7,
 7,
 4,
 9,
 7,
 0,
 2,
 2,
 5,
 7,
 2,
 4,
 7,
 1,
 5,
 0,
 5,
 9,
 2,
 6,
 0,
 1,
 5,
 1,
 5,
 7,
 7,
 9,
 5,
 7,
 7,
 6,
 6,
 9,
 8,
 3,
 6,
 0,
 7,
 9,
 3,
 0,
 8,
 6,
 3,
 1,
 3,


In [90]:
output_df = pd.DataFrame(predictions, columns = ['label'])
output_df.to_csv('./Output/polysvm.csv', sep=",", float_format='%d', index_label="id")